In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Load data
data = pd.read_csv('dataset.csv') 



In [5]:
# Encode categorical features
name_mapping = {Name: i for i, Name in enumerate(data['Name'].unique())}
places_mapping = {Place: i for i, Place in enumerate(data['Place'].unique())}
city_mapping = {City: i for i, City in enumerate(data['City'].unique())}
category1_mapping = {Category: i for i, Category in enumerate(data['Category1'].unique())}
category2_mapping = {Category: i for i, Category in enumerate(data['Category2'].unique())}

data['Name'] = data['Name'].map(name_mapping)
data['Place'] = data['Place'].map(places_mapping)
data['City'] = data['City'].map(city_mapping)
data['Category1'] = data['Category1'].map(category1_mapping)
data['Category2'] = data['Category2'].map(category2_mapping)

num_users = len(data['Name'].unique())
num_places = len(data['Place'].unique())



In [6]:
# Create user-item ratings matrix
ratings_matrix = np.zeros((num_users, num_places))
for row in data.itertuples():
    ratings_matrix[row.Name, row.Place] = row.Rating

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(num_places,)),
    tf.keras.layers.Dense(num_places)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(ratings_matrix, ratings_matrix, epochs=50)



Epoch 1/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1182
Epoch 2/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1065
Epoch 3/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1040
Epoch 4/50
625/625 [==============================] - 1s 2ms/step - loss: 0.1031
Epoch 5/50
625/625 [==============================] - 1s 2ms/step - loss: 0.1027
Epoch 6/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1026
Epoch 7/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1025
Epoch 8/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1024
Epoch 9/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1023
Epoch 10/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1023
Epoch 11/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1022
Epoch 12/50
625/625 [==============================] - 1s 1ms/step - loss: 0.1022
Epoch 13/50
625/625 [====

In [7]:
# Generate recommendations
user_ids = [0, 1, 2, 3]  
recommendations = model.predict(ratings_matrix)

# Print the recommendations
for user_id, recs in zip(user_ids, recommendations):
    print(f"User ID: {user_id}")
    print("Recommendations:")
    rec_indices = np.argsort(recs)[::-1]
    for i in range(3):  # Get top 3 recommendations
        place_id = rec_indices[i]
        rating = recs[place_id]
        place_name = list(places_mapping.keys())[list(places_mapping.values()).index(place_id)]
        print(f"Place: {place_name}, Rating: {rating}")
    print()

625/625 [==============================] - 1s 1ms/step
User ID: 0
Recommendations:
Place: Museum Kereta Ambarawa, Rating: 0.07434378564357758
Place: Semarang Chinatown, Rating: 0.07094637304544449
Place: Tebing Breksi, Rating: 0.06886350363492966

User ID: 1
Recommendations:
Place: Museum Kereta Ambarawa, Rating: 0.07434378564357758
Place: Semarang Chinatown, Rating: 0.07094637304544449
Place: Tebing Breksi, Rating: 0.06886350363492966

User ID: 2
Recommendations:
Place: Museum Kereta Ambarawa, Rating: 0.07434378564357758
Place: Semarang Chinatown, Rating: 0.07094637304544449
Place: Tebing Breksi, Rating: 0.06886350363492966

User ID: 3
Recommendations:
Place: Museum Kereta Ambarawa, Rating: 0.07434378564357758
Place: Semarang Chinatown, Rating: 0.07094637304544449
Place: Tebing Breksi, Rating: 0.06886350363492966



In [1]:
!pip install tflite

     ------------------------------------- 123.6/123.6 kB 65.9 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
export_dir = 'saved_model'
tf.saved_model.save(model,export_dir)

INFO:tensorflow:Assets written to: saved_model\assets


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()
